<a href="https://colab.research.google.com/github/alexander-harmaty/Breast-Cancer-Prognosis-Prediction/blob/main/fusion_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/" alt="Open In Colab"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Environment Setup and Imports**

In [1]:
import pickle
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import (
    Dense, Dropout, BatchNormalization,
    Concatenate, Multiply, Activation
)


# **Data Preprocessing**

## *Data Loading*

In [2]:
# --- Data Loading & Alignment ---
with open('rnn_features.pkl','rb') as f: rnn = pickle.load(f)
with open('cnn_features.pkl','rb') as f: cnn = pickle.load(f)

df_rnn = pd.DataFrame(rnn['features'], index=rnn['ids'])
df_rnn['label'] = rnn['labels']
df_cnn = pd.DataFrame(cnn['features'], index=cnn['ids'])
df_cnn['label'] = cnn['labels']

common = df_rnn.index.intersection(df_cnn.index)
df_rnn = df_rnn.loc[common]
df_cnn = df_cnn.loc[common]

X_rnn = df_rnn.drop(columns='label').values
X_cnn = df_cnn.drop(columns='label').values
y     = df_rnn['label'].values

print('Feature shapes:', 'CNN:', X_cnn.shape, 'RNN:', X_rnn.shape)
print('Label distribution:', Counter(y))


Feature shapes: CNN: (100, 128) RNN: (100, 32)
Label distribution: Counter({np.float64(0.0): 90, np.float64(1.0): 10})


## *Feature Merging*
single-input vs dual-input (choose which one to run)

plan:

*   single‑input fusion for baseline  
*   dual‑input gated fusion for fine tuning

### dual-input

## *Data Splitting*

In [3]:
# --- Train/Validation/Test Split ---
Xc_train, Xc_temp, Xr_train, Xr_temp, y_train, y_temp = train_test_split(
    X_cnn, X_rnn, y,
    test_size=0.30, random_state=42, stratify=y
)
try:
    Xc_val, Xc_test, Xr_val, Xr_test, y_val, y_test = train_test_split(
        Xc_temp, Xr_temp, y_temp,
        test_size=0.50, random_state=42, stratify=y_temp
    )
except ValueError:
    Xc_val, Xc_test, Xr_val, Xr_test, y_val, y_test = train_test_split(
        Xc_temp, Xr_temp, y_temp,
        test_size=0.50, random_state=42
    )
print('Split sizes:', Counter(y_train), Counter(y_val), Counter(y_test))


Split sizes: Counter({np.float64(0.0): 63, np.float64(1.0): 7}) Counter({np.float64(0.0): 13, np.float64(1.0): 2}) Counter({np.float64(0.0): 14, np.float64(1.0): 1})


# **Fusion Model**

## *Model Building*

In [4]:
# --- Build gated fusion model ---
input_cnn = Input(shape=(X_cnn.shape[1],), name='cnn_in')
input_rnn = Input(shape=(X_rnn.shape[1],), name='rnn_in')

# Modality-specific projections
proj_cnn = Dense(128, activation='relu')(input_cnn)
proj_cnn = BatchNormalization()(proj_cnn)
proj_cnn = Dropout(0.3)(proj_cnn)

proj_rnn = Dense(128, activation='relu')(input_rnn)
proj_rnn = BatchNormalization()(proj_rnn)
proj_rnn = Dropout(0.3)(proj_rnn)

# Gating mechanism
gate = Concatenate()([proj_cnn, proj_rnn])
gate = Dense(128, activation='sigmoid', name='fusion_gate')(gate)

# Elementwise fusion
fused_cnn = Multiply()([gate, proj_cnn])
inv_gate = Activation('linear')(1.0 - gate)
fused_rnn = Multiply()([inv_gate, proj_rnn])
fused = tf.keras.layers.Add()([fused_cnn, fused_rnn])

# Joint MLP
x = Dense(64, activation='relu')(fused)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid', name='out')(x)

model = Model(inputs=[input_cnn, input_rnn], outputs=output, name='gated_fusion_model')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
model.summary()


Model: "gated_fusion_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ cnn_in (InputLayer) │ (None, 128)       │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rnn_in (InputLayer) │ (None, 32)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     16,512 │ cnn_in[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │      4,224 │ rnn_in[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128)       │        512 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ dropout[0][0],    │
│ (Concatenate)       │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_gate (Dense) │ (None, 128)       │     32,896 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ subtract (Subtract) │ (None, 128)       │          0 │ fusion_gate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 128)       │          0 │ subtract[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 128)       │          0 │ fusion_gate[0][0… │
│                     │                   │            │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 128)       │          0 │ activation[0][0], │
│ (Multiply)          │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 128)       │          0 │ multiply[0][0],   │
│                     │                   │            │ multiply_1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      8,256 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_2[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64)        │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      2,080 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 32)        │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out (Dense)         │ (None, 1)         │         33 │ dropout_3[0][0] 

 Total params: 65,281 (255.00 KB)

 Trainable params: 64,641 (252.50 KB)

 Non-trainable params: 640 (2.50 KB)

## *Model Training*

In [5]:
# --- Training ---
history = model.fit(
    {'cnn_in': Xc_train, 'rnn_in': Xr_train},
    y_train,
    validation_data=({'cnn_in': Xc_val, 'rnn_in': Xr_val}, y_val),
    epochs=50, batch_size=32, verbose=2
)


Epoch 1/50
3/3 - 6s - 2s/step - accuracy: 0.5714 - auc: 0.5737 - loss: 0.7059 - val_accuracy: 0.9333 - val_auc: 1.0000 - val_loss: 0.6001
Epoch 2/50
3/3 - 1s - 236ms/step - accuracy: 0.7000 - auc: 0.6678 - loss: 0.6865 - val_accuracy: 1.0000 - val_auc: 1.0000 - val_loss: 0.5789
Epoch 3/50
3/3 - 0s - 70ms/step - accuracy: 0.5857 - auc: 0.8549 - loss: 0.6599 - val_accuracy: 1.0000 - val_auc: 1.0000 - val_loss: 0.5591
Epoch 4/50
3/3 - 0s - 98ms/step - accuracy: 0.7429 - auc: 0.8832 - loss: 0.4597 - val_accuracy: 1.0000 - val_auc: 1.0000 - val_loss: 0.5394
Epoch 5/50
3/3 - 0s - 59ms/step - accuracy: 0.7571 - auc: 0.8764 - loss: 0.4769 - val_accuracy: 1.0000 - val_auc: 1.0000 - val_loss: 0.5211
Epoch 6/50
3/3 - 0s - 107ms/step - accuracy: 0.8429 - auc: 0.9830 - loss: 0.3766 - val_accuracy: 1.0000 - val_auc: 1.0000 - val_loss: 0.5036
Epoch 7/50
3/3 - 0s - 59ms/step - accuracy: 0.9429 - auc: 1.0000 - loss: 0.2549 - val_accuracy: 1.0000 - val_auc: 1.0000 - val_loss: 0.4879
Epoch 8/50
3/3 - 0s 

## *Model Evaluation*

In [6]:
# --- Evaluation on Test Set ---
y_test_prob = model.predict({'cnn_in': Xc_test, 'rnn_in': Xr_test}).ravel()
y_test_pred = (y_test_prob >= 0.5).astype(int)
print('Confusion matrix on test set:')
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))
print('Test ROC AUC:', roc_auc_score(y_test, y_test_prob))

# --- Stratified K-Fold Cross-Validation ---
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
auc_scores = []
for train_idx, test_idx in skf.split(X_cnn, y):
    Xc_tr, Xc_te = X_cnn[train_idx], X_cnn[test_idx]
    Xr_tr, Xr_te = X_rnn[train_idx], X_rnn[test_idx]
    y_tr, y_te = y[train_idx], y[test_idx]
    # Reinitialize model for each fold
    m = tf.keras.models.clone_model(model)
    m.compile(optimizer='adam', loss='binary_crossentropy')
    m.fit(
        {'cnn_in': Xc_tr, 'rnn_in': Xr_tr},
        y_tr,
        epochs=30, batch_size=32, verbose=0
    )
    prob = m.predict({'cnn_in': Xc_te, 'rnn_in': Xr_te}).ravel()
    auc_scores.append(roc_auc_score(y_te, prob))
print('Stratified K-Fold mean ROC AUC:', np.mean(auc_scores))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
Confusion matrix on test set:
[[14  0]
 [ 0  1]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        14
         1.0       1.00      1.00      1.00         1

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15

Test ROC AUC: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
Stratified K-Fold mean ROC AUC: 1.0
